<a href="https://colab.research.google.com/github/amalrik1/develop_ml_course/blob/main/practice2/adult_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пример демонстрирует импорт, предобработку и построение модели машинного обучения для датасета [Adult](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset).

Итоговая модель сохраняется как сериализованный pickle-объект.

Датасет, модель и эксперимент загружаются в ClearML.
Авторизационные ключи в облако ClearML должны быть ваши личные.

In [1]:
%env CLEARML_WEB_HOST=http://10.200.1.43:8080/
%env CLEARML_API_HOST=http://10.200.1.43:8008
%env CLEARML_FILES_HOST=http://10.200.1.43:8081
%env CLEARML_API_ACCESS_KEY=ZX5U64S5N72PZOXRZA5Q5OBJ6689QX
%env CLEARML_API_SECRET_KEY=WaURJ_NjFCoZPQyDDrUU41LgwowqFX-slFNmSr5rER1C0catHTC8RIsxaAyARQ0Cr5s

env: CLEARML_WEB_HOST=http://10.200.1.43:8080/
env: CLEARML_API_HOST=http://10.200.1.43:8008
env: CLEARML_FILES_HOST=http://10.200.1.43:8081
env: CLEARML_API_ACCESS_KEY=ZX5U64S5N72PZOXRZA5Q5OBJ6689QX
env: CLEARML_API_SECRET_KEY=WaURJ_NjFCoZPQyDDrUU41LgwowqFX-slFNmSr5rER1C0catHTC8RIsxaAyARQ0Cr5s


In [7]:
from clearml import Task, Logger
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Чтобы начать работать с ClearML инициируем новую сессию:
task = Task.init(
    project_name='Adult model', 
    task_name='Эксперимент  1', 
    tags=['DecisionTreeClassifier'])

ClearML Task: created new task id=00c47c625913411983f214fd895e6130
2024-10-09 17:45:09,747 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-10-09 17:45:09,755 - clearml.Repository Detection - WARNING - Please install nbconvert using "pip install nbconvert"
ClearML results page: http://10.200.1.43:8080/projects/2ed7f03f23174e768dca606caeaeae7c/experiments/00c47c625913411983f214fd895e6130/output/log
2024-10-09 17:45:10,526 - clearml.Task - INFO - Storing jupyter notebook directly as code
CLEARML-SERVER new package available: UPGRADE to v1.16.2 is recommended!
Release Notes:
### Bug Fixes
- Fix no graphs are shown in workers and queues screens


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
2024-10-09 17:48:51,475 - clearml.storage - INFO - Uploading: 5.50MB to d:/code/polytex/4/adults.csv


                                             0% | 0.00/5.5 MB [00:00<?, ?MB/s]: d:\code\polytex\.venv\Lib\site-packages\tqdm\std.py:636: TqdmWarning:

clamping frac to range [0, 1]

██████████████████████████████████ 100% | 5.50/5.5 MB [00:00<00:00, 49.14MB/s]: 


Загрузим датасет
Помимо загрузки данных в pandas, мы попутно отправляем наши данные в ClearML как Артефакт, просто указав путь к CSV-файлу и задав имя артифакта. Все загруженные артефакты можно найти на вкладке Artifacts, соответствующего эксперимента.

In [8]:

fpath = 'adults.csv'
df_raw = pd.read_csv(fpath)
task.upload_artifact(name='data.raw', artifact_object=fpath)

True

In [9]:
df_raw.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


ClearML может хранить в виде Артифактов практически любые Python-объекты. Здесь мы поместили в Артефакт вывод пандосовского метода describe. 

In [10]:
task.upload_artifact(
    name='eda.describe.object', 
    artifact_object=df_raw.describe(include=object))
task.upload_artifact(
    name='eda.describe.number', 
    artifact_object=df_raw.describe(include=np.number))

True

In [13]:
df = df.drop(['fnlwgt', 'education_num'], axis=1)

In [14]:
col_names = df.columns

for c in col_names:
    df = df.replace("?", numpy.NaN)
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [15]:
df.replace(['Divorced', 'Married-AF-spouse',
            'Married-civ-spouse', 'Married-spouse-absent',
            'Never-married', 'Separated', 'Widowed'],
           ['divorced', 'married', 'married', 'married',
            'not married', 'not married', 'not married'], inplace=True)

category_col = ['workclass', 'race', 'education', 'marital_status', 'occupation',
                'relationship', 'sex', 'native_country', 'class']
labelEncoder = preprocessing.LabelEncoder()

mapping_dict = {}
for col in category_col:
    df[col] = labelEncoder.fit_transform(df[col])

    le_name_mapping = dict(zip(labelEncoder.classes_,
                               labelEncoder.transform(labelEncoder.classes_)))

    mapping_dict[col] = le_name_mapping
print(mapping_dict)

{'workclass': {' ?': 0, ' Federal-gov': 1, ' Local-gov': 2, ' Never-worked': 3, ' Private': 4, ' Self-emp-inc': 5, ' Self-emp-not-inc': 6, ' State-gov': 7, ' Without-pay': 8}, 'race': {' Amer-Indian-Eskimo': 0, ' Asian-Pac-Islander': 1, ' Black': 2, ' Other': 3, ' White': 4}, 'education': {' 10th': 0, ' 11th': 1, ' 12th': 2, ' 1st-4th': 3, ' 5th-6th': 4, ' 7th-8th': 5, ' 9th': 6, ' Assoc-acdm': 7, ' Assoc-voc': 8, ' Bachelors': 9, ' Doctorate': 10, ' HS-grad': 11, ' Masters': 12, ' Preschool': 13, ' Prof-school': 14, ' Some-college': 15}, 'marital_status': {' Divorced': 0, ' Married-AF-spouse': 1, ' Married-civ-spouse': 2, ' Married-spouse-absent': 3, ' Never-married': 4, ' Separated': 5, ' Widowed': 6}, 'occupation': {' ?': 0, ' Adm-clerical': 1, ' Armed-Forces': 2, ' Craft-repair': 3, ' Exec-managerial': 4, ' Farming-fishing': 5, ' Handlers-cleaners': 6, ' Machine-op-inspct': 7, ' Other-service': 8, ' Priv-house-serv': 9, ' Prof-specialty': 10, ' Protective-serv': 11, ' Sales': 12, '

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = df.values[:, 0:12]
Y = df.values[:, 12]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
           X, Y, test_size = 0.3, random_state = 100)

dt_clf_gini = DecisionTreeClassifier(criterion = "gini",
                                     random_state = 100,
                                     max_depth = 5,
                                     min_samples_leaf = 5)

dt_clf_gini.fit(X_train, y_train)
y_pred_gini = dt_clf_gini.predict(X_test)

print ("Decision Tree using Gini Index\nAccuracy is ",
             accuracy_score(y_test, y_pred_gini)*100 )

Decision Tree using Gini Index
Accuracy is  82.5769467003344


In [18]:
import pickle

In [19]:
with open("model.pkl", "wb") as f:
    pickle.dump(dt_clf_gini, f)